In [1]:
import pandas as pd
import numpy as np

CUSTOMER = pd.read_csv('lv2-2404.csv')
print(CUSTOMER.shape)
print(CUSTOMER.info())

(900, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 11 columns):
CUS_ID     900 non-null int64
REGION     900 non-null int64
TENURE     900 non-null int64
AGE        900 non-null int64
INCOME     900 non-null int64
MARITAL    900 non-null int64
ED         900 non-null int64
EMPLOY     900 non-null int64
RETIRE     900 non-null int64
GENDER     900 non-null int64
CUSTCAT    900 non-null object
dtypes: int64(10), object(1)
memory usage: 77.5+ KB
None


### Q1

#### Step 1-1

In [2]:
df1 = CUSTOMER.copy()
df_rg1 = df1.loc[df1['REGION']==1, 'TENURE'].copy()
df_rg2 = df1.loc[df1['REGION']==2, 'TENURE'].copy()
df_rg3 = df1.loc[df1['REGION']==3, 'TENURE'].copy()
print(df_rg1.shape, df_rg2.shape, df_rg3.shape)

(291,) (296,) (313,)


#### Step 1-2

In [3]:
num_mean = df_rg1.mean()
num_std = df_rg1.std()

a_low = num_mean - 1.968178*num_std/np.sqrt(len(df_rg1))
a_up = num_mean + 1.968178*num_std/np.sqrt(len(df_rg1))

num_mean = df_rg2.mean()
num_std = df_rg2.std()

b_low = num_mean - 1.968038*num_std/np.sqrt(len(df_rg2))
b_up = num_mean + 1.968038*num_std/np.sqrt(len(df_rg2))

num_mean = df_rg3.mean()
num_std = df_rg3.std()

c_low = num_mean - 1.967596*num_std/np.sqrt(len(df_rg3))
c_up = num_mean + 1.967596*num_std/np.sqrt(len(df_rg3))

#### Step 1-3

In [4]:
A = np.max([a_up, b_up, c_up]) + np.min([a_low, b_low, c_low])
print(A)

71.18508592024864


In [5]:
ans21 = round(A, 2)
print('ans1:',ans21)

ans1: 71.19


### Q2

#### Step 2-1

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

df2 = CUSTOMER.copy()
VAR2 = ['TENURE','AGE','INCOME','EMPLOY']
VAR2s = [s+'_S' for s in VAR2]
df2s = pd.DataFrame(StandardScaler().fit_transform(df2[VAR2]), columns = VAR2s)
df2c = pd.concat([df2, df2s], axis=1)
print(df2c.shape)

(900, 15)


#### Step 2-2

In [7]:
df2c['clus'] = KMeans(random_state = 1234, n_clusters=4).fit_predict(df2c[VAR2s])
df2c

,CUS_ID,REGION,TENURE,AGE,INCOME,MARITAL,ED,EMPLOY,RETIRE,GENDER,CUSTCAT,TENURE_S,AGE_S,INCOME_S,EMPLOY_S,clus
0,1,1,67,40,59,1,3,11,0,0,Class C,1.472498,-0.128495,-0.160074,0.009789,0
1,2,1,50,43,27,0,3,4,0,0,Class C,0.677340,0.109296,-0.454744,-0.683155,0
2,3,1,3,24,20,1,2,3,0,1,Class C,-1.521039,-1.396714,-0.519203,-0.782147,2
3,4,1,40,42,108,1,3,17,0,0,Class B,0.209600,0.030032,0.291141,0.603741,0
4,5,2,48,35,50,0,1,16,0,0,Class C,0.583792,-0.524814,-0.242950,0.504749,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,896,1,45,52,46,0,5,8,0,1,Class B,0.443470,0.822669,-0.279784,-0.287187,0
896,897,1,8,29,21,1,5,0,0,0,Class A,-1.287169,-1.000396,-0.509995,-1.079123,2
897,898,1,29,32,49,1,4,3,0,1,Class B,-0.304915,-0.762605,-0.252158,-0.782147,2
898,899,3,20,48,24,1,2,3,0,0,Class D,-0.725881,0.505614,-0.482370,-0.782147,2


#### Step 2-3

In [8]:
cnt1 = df2c[df2c['clus']==0][VAR2s].mean(axis=0)
cnt2 = df2c[df2c['clus']==1][VAR2s].mean(axis=0)
cnt3 = df2c[df2c['clus']==2][VAR2s].mean(axis=0)
cnt4 = df2c[df2c['clus']==3][VAR2s].mean(axis=0)

#### Step 2-4

In [9]:
arr_std = np.zeros(4)
for i in [0, 1, 2, 3]:
    df_tmp = df2c[df2c['clus']==i][VAR2s].copy()
    cnti = df2c[df2c['clus']==i][VAR2s].mean(axis=0)
    udis2 = (df_tmp['TENURE_S'] - cnti['TENURE_S'])**2 +\
    (df_tmp['AGE_S'] - cnti['AGE_S'])**2 +\
    (df_tmp['INCOME_S'] - cnti['INCOME_S'])**2 +\
    (df_tmp['EMPLOY_S'] - cnti['EMPLOY_S'])**2
    arr_std[i] = udis2.map(lambda x: np.sqrt(x)).std()
arr_std

array([0.43974249, 0.60146725, 0.3949086 , 1.99563486])

#### Step 2-5

In [10]:
std_min = arr_std.min()
print(std_min)

0.39490860093944474


In [11]:
ans22 = round(std_min, 3)
print('ans2:',ans22)

ans2: 0.395


### Q3

#### Step 3-1

In [12]:
df3 = CUSTOMER.copy()
df23 = df3[df3['CUSTCAT'].isin(['Class A','Class D'])].copy()

#### Step 3-2

In [13]:
df23['AGE_GROUP'] = 'AGE_50'
df23.loc[df23['AGE']<50, 'AGE_GROUP'] = 'AGE_40'
df23.loc[df23['AGE']<40, 'AGE_GROUP'] = 'AGE_30'
df23.loc[df23['AGE']<30, 'AGE_GROUP'] = 'AGE_20'
df23['AGE_GROUP'].value_counts()

AGE_30    136
AGE_40    106
AGE_20    104
AGE_50    103
Name: AGE_GROUP, dtype: int64

#### Step 3-3

In [14]:
list_fts = ['REGION','ED','AGE_GROUP']
df_dum = pd.get_dummies(df23.loc[:, list_fts].astype(str), drop_first=True)
df23d = pd.concat([df23, df_dum], axis=1)

#### Step 3-4

In [15]:
train23 = df23d[df23d['CUS_ID']%5 != 0].copy()
test23 = df23d[df23d['CUS_ID']%5 == 0].copy()
print(train23.shape, test23.shape)

(356, 21) (93, 21)


#### Step 3-5

In [16]:
from sklearn.tree import DecisionTreeClassifier

DT00 = DecisionTreeClassifier(random_state=1234, max_depth=3)
list_fts = ['TENURE','INCOME','MARITAL','EMPLOY','RETIRE','GENDER'] + list(df_dum.columns)
DT00.fit(train23.loc[:, list_fts], train23['CUSTCAT'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1234, splitter='best')

#### Step 3-6

In [17]:
arr_pred = DT00.predict(test23.loc[:, list_fts])
arr_act = np.array(test23['CUSTCAT'])
precision = ((arr_pred=='Class A') & (arr_act=='Class A')).sum()/(arr_pred=='Class A').sum()
print(precision)

0.578125


In [18]:
ans23 = round(precision, 3)
print('ans3:',ans23)

ans3: 0.578


In [19]:
print(ans21)
print(ans22)
print(ans23)

71.19
0.395
0.578
